This is a script that removed ~108 outlier samples from the data set. 

In [1]:
library(tidyverse)

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1       ✔ purrr   0.3.2  
✔ tibble  2.1.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.3       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
is_gt8 <- function(dist, d){
    samples <- rownames(d)[dist > 8 | dist < -8]
    return(samples)
}

zap_outliers <- function(data){
    d <- as.data.frame(scale(data %>% select(B11:Cd111), center=T, scale=T))
    rownames(d) <- data$samplename
    x <- apply(d, 2, is_gt8, d=d)
    outliers <- unique(unlist(x))
    d_clean <- data[!(data$samplename %in% outliers),]
    rownames(d_clean) <- d_clean$samplename
    return(d_clean) 
}

In [3]:
data <- read.csv('./df/vitis_ionomics_data.csv')
data$Li7 <- NULL
data$X <- NULL
dim(data)

[1] 7479   21

In [4]:
dupes <- as.data.frame(table(data$samplename)[table(data$samplename) > 1])
d <- data[!(data$samplename %in% dupes$Var1),]

dim(d)

[1] 7439   21

In [5]:
d <- zap_outliers(d)
dim(d)

[1] 7331   21

In [6]:
summary(d)

                  samplename        B11              Na23       
 2017_anthesis_10_A_1_X:   1   Min.   :-22.57   Min.   :-20.11  
 2017_anthesis_10_A_1_Y:   1   1st Qu.: 14.50   1st Qu.: 14.10  
 2017_anthesis_10_A_1_Z:   1   Median : 18.85   Median : 23.11  
 2017_anthesis_10_A_2_X:   1   Mean   : 21.88   Mean   : 36.21  
 2017_anthesis_10_A_2_Y:   1   3rd Qu.: 26.43   3rd Qu.: 39.53  
 2017_anthesis_10_A_2_Z:   1   Max.   :123.29   Max.   :450.75  
 (Other)               :7325                                    
      Mg26              Al27             P31               S34          
 Min.   : -702.1   Min.   :-17.11   Min.   :   60.5   Min.   :-56835.3  
 1st Qu.: 2022.6   1st Qu.: 10.04   1st Qu.: 1569.1   1st Qu.:   829.6  
 Median : 2536.5   Median : 17.98   Median : 2452.1   Median :  1478.6  
 Mean   : 2798.6   Mean   : 21.22   Mean   : 2728.4   Mean   :  1602.9  
 3rd Qu.: 3332.3   3rd Qu.: 27.89   3rd Qu.: 3482.7   3rd Qu.:  2085.5  
 Max.   :13035.7   Max.   :186.17   Max.  

In [7]:
write.csv(d, "./df/vitis_ionomics_data_noOutlier.csv")